# Generative Adversarial Network (GAN)

## Concept

GAN is composed of two neural networks that compete with each other, hence the name adversarial. There's the **Generator** network (G) and **Discriminator** network (D).

### Generator Network

The generator's task is to generate _fake data_ that could, hopefully, fool the discriminator. The generator does not have access to the _real data_ that the discriminator expects, instead it improves its ability to generate fake data by learning what feature of the data is expected by the discriminator.

- Input: vector of **noise**
- Output: fake data with the **same shape** as the real data

Stochastic gradient for generator network:

<center> $\bigtriangledown_{\theta_{d}}\frac{1}{m} \displaystyle\sum_{i=1}^{m}log (1-D(G(z^{(i)})))$ </center>

The target is to **minimize** the value of $1-D(G(z^{(i)}))$ which means **maximizing** the value of $D(G(z^{(i)})$, the probability that the discriminator passes the fake data generated by the generator as real.

### Discriminator Network

The discriminator's task is to determine if a data is _real_ or _fake_. The discriminator is actually updated by **ascending** its stochastic gradient, instead of descending.

- Input: real and fake **data**
- Output: binary (is the data real or fake)

Stochastic gradient for discriminator network:

<center> $\bigtriangledown_{\theta_{d}}\frac{1}{m} \displaystyle\sum_{i=1}^{m}[log D(x^{(i)}) + log (1-D(G(z^{(i)})))]$ </center>

## GAN for MNIST Dataset with Keras

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import sys

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

class GAN():
    
    def __init__(self):
        self.img_rows = 28 
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        
        self.noise_shape = (100,)

        # Manually tune down learning rate to avoid oscillation
        optimizer = Adam(lr=0.0002, beta_1=0.5)

        # -------------
        # Discriminator
        # -------------
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', 
                                   optimizer=optimizer,
                                   metrics=['accuracy'])
        # ---------
        # Generator
        # ---------
        self.generator = self.build_generator()
        self.generator.compile(loss='binary_crossentropy',
                               optimizer=optimizer)
        # --------
        # Combined
        # --------
        # The combined model is created by stacking generator and discriminator.
        # Noise ---Generator--> Generated Image ---Discriminator--> Validity
        
        z = Input(shape=self.noise_shape)
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False
        
        validity = self.discriminator(img)

        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy',
                              optimizer=optimizer)

        
    def build_discriminator(self):
        model = Sequential()
        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        
        model.summary()

        # Discriminator takes an image as an input and outputs its validity
        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

        
    def build_generator(self):
        # BatchNormalization maintains the mean activation close to 0
        # and the activation standard deviation close to 1
        model = Sequential()
        model.add(Dense(256, input_shape=self.noise_shape))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='sigmoid'))
        model.add(Reshape(self.img_shape))
        
        model.summary()

        # Generator takes noise as an input and outputs an image
        noise = Input(shape=self.noise_shape)
        img = model(noise)

        return Model(noise, img)

    
    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()
        
        # X_train shape: (m, W, H)
        
        # Rescale 0 to 1
        X_train = (X_train.astype(np.float32)) / 255
        
        # Change X_train's shape to (m, W, H, c)
        X_train = np.expand_dims(X_train, axis=3)

        half_batch = int(batch_size / 2)

        for epoch in range(epochs):

            # -------------------
            # Train Discriminator
            # -------------------
            # Select a random half batch of images
            idx = np.random.randint(0, X_train.shape[0], half_batch)
            imgs = X_train[idx]

            # Generate a half batch of new images
            noise = np.random.normal(0, 1, (half_batch, 100))
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------
            # Train Generator
            # ---------------
            noise = np.random.normal(0, 1, (batch_size, 100))

            # The generator wants to fool the discriminator, hence trained with valid label (1)
            valid_y = np.array([1] * batch_size)

            # Train the generator
            g_loss = self.combined.train_on_batch(noise, valid_y)


            # Print progress
            print ("{:5d} [D loss: {}, acc_real: {:2f}, acc_fake: {:2f}] [G loss: {}]".format(epoch, d_loss[0], 100*d_loss_real[1], 100*d_loss_fake[1], g_loss))

            # Save generated samples at save_interval
            if epoch == 0 or (epoch + 1) % save_interval == 0:
                r, c = 3, 3

                fig, axs = plt.subplots(r, c)
                
                prediction = self.discriminator.predict(gen_imgs)
                prediction = np.round(prediction)
                
                fooled_indices = []
                
                for i in range(len(prediction)):
                    if prediction[i] == 1:
                        fooled_indices.append(i)
                
                fooled_count = len(fooled_indices)
                
                # Export generated images that the discriminator classified as real
                cnt = 0
                if (fooled_count > 0):
                    for i in range(r):
                        for j in range(c):
                            # Prevent error where count exceeds the number of failed test case for discriminator
                            if (cnt >= fooled_count):
                                break
                            axs[i,j].imshow(gen_imgs[fooled_indices[cnt], :,:,0], cmap='gray')
                            axs[i,j].axis('off')
                            cnt += 1
                            
                fig.savefig("gan/images/mnist_{}_fooled.png".format(epoch+1))
                plt.close()
                
                # Export generated images that the discriminator classified as fake
                fig, axs = plt.subplots(r, c)
                success_count = len(prediction) - fooled_count
                cnt = 0
                if (success_count > 0):
                    for i in range(r):
                        for j in range(c):
                            # Find index not in faied_indices
                            while cnt in fooled_indices:
                                cnt += 1
                                
                            # Prevent error where count exceeds the number of successful test case for discriminator
                            if (cnt >= success_count):
                                break
                            
                            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                            axs[i,j].axis('off')
                            cnt += 1
                fig.savefig("gan/images/mnist_{}_detected.png".format(epoch+1))
                plt.close()


if __name__ == '__main__':
    gan = GAN()
    gan.train(epochs=30000, batch_size=64, save_interval=250)

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________
____

/home/rionaldichandraseta/miniconda3/envs/gan-image/lib/python3.6/site-packages/keras/engine/training.py:478: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


    0 [D loss: 0.8570461273193359, acc_real: 12.500000, acc_fake: 0.000000] [G loss: 0.9117695093154907]
    1 [D loss: 0.5363436937332153, acc_real: 62.500000, acc_fake: 100.000000] [G loss: 1.6201791763305664]
    2 [D loss: 0.4299195110797882, acc_real: 65.625000, acc_fake: 100.000000] [G loss: 2.2145326137542725]
    3 [D loss: 0.34796568751335144, acc_real: 96.875000, acc_fake: 100.000000] [G loss: 2.560175895690918]
    4 [D loss: 0.31124112010002136, acc_real: 93.750000, acc_fake: 100.000000] [G loss: 2.7586584091186523]
    5 [D loss: 0.2329246699810028, acc_real: 100.000000, acc_fake: 100.000000] [G loss: 2.8853535652160645]
    6 [D loss: 0.18145573139190674, acc_real: 100.000000, acc_fake: 100.000000] [G loss: 3.090276002883911]
    7 [D loss: 0.1371222287416458, acc_real: 100.000000, acc_fake: 100.000000] [G loss: 3.3503031730651855]
    8 [D loss: 0.11620654165744781, acc_real: 100.000000, acc_fake: 100.000000] [G loss: 3.5724642276763916]
    9 [D loss: 0.0829872339963913

   77 [D loss: 0.41016876697540283, acc_real: 87.500000, acc_fake: 68.750000] [G loss: 7.791046142578125]
   78 [D loss: 0.4195091128349304, acc_real: 93.750000, acc_fake: 62.500000] [G loss: 8.157245635986328]
   79 [D loss: 0.32739728689193726, acc_real: 75.000000, acc_fake: 100.000000] [G loss: 8.333313941955566]
   80 [D loss: 0.33897483348846436, acc_real: 90.625000, acc_fake: 84.375000] [G loss: 7.997773170471191]
   81 [D loss: 0.30848658084869385, acc_real: 93.750000, acc_fake: 78.125000] [G loss: 7.061298370361328]
   82 [D loss: 0.40038156509399414, acc_real: 93.750000, acc_fake: 65.625000] [G loss: 8.442131042480469]
   83 [D loss: 0.3236226737499237, acc_real: 90.625000, acc_fake: 87.500000] [G loss: 8.150056838989258]
   84 [D loss: 0.4552578032016754, acc_real: 78.125000, acc_fake: 75.000000] [G loss: 6.985980987548828]
   85 [D loss: 0.4969145655632019, acc_real: 90.625000, acc_fake: 53.125000] [G loss: 7.927091598510742]
   86 [D loss: 0.2641291916370392, acc_real: 90.6

...

29668 [D loss: 0.5872489809989929, acc_real: 56.250000, acc_fake: 87.500000] [G loss: 1.0759122371673584]
29669 [D loss: 0.5982670783996582, acc_real: 53.125000, acc_fake: 75.000000] [G loss: 0.933054506778717]
29670 [D loss: 0.5890692472457886, acc_real: 59.375000, acc_fake: 71.875000] [G loss: 0.9591022729873657]
29671 [D loss: 0.6836186647415161, acc_real: 50.000000, acc_fake: 53.125000] [G loss: 0.9150797724723816]
29672 [D loss: 0.6542816162109375, acc_real: 53.125000, acc_fake: 62.500000] [G loss: 0.9999942779541016]
29673 [D loss: 0.6369497179985046, acc_real: 71.875000, acc_fake: 65.625000] [G loss: 0.9970570802688599]
29674 [D loss: 0.6949458122253418, acc_real: 31.250000, acc_fake: 68.750000] [G loss: 0.9307000637054443]
29675 [D loss: 0.6244706511497498, acc_real: 68.750000, acc_fake: 59.375000] [G loss: 0.8663560748100281]
29676 [D loss: 0.634258508682251, acc_real: 78.125000, acc_fake: 53.125000] [G loss: 1.0014748573303223]
29677 [D loss: 0.614585280418396, acc_real: 68.7

29746 [D loss: 0.6439293026924133, acc_real: 62.500000, acc_fake: 71.875000] [G loss: 0.9360356330871582]
29747 [D loss: 0.6260690093040466, acc_real: 62.500000, acc_fake: 75.000000] [G loss: 0.9451627731323242]
29748 [D loss: 0.5662533044815063, acc_real: 90.625000, acc_fake: 65.625000] [G loss: 0.9617828726768494]
29749 [D loss: 0.6579138040542603, acc_real: 59.375000, acc_fake: 68.750000] [G loss: 1.0070219039916992]
29750 [D loss: 0.6606680154800415, acc_real: 50.000000, acc_fake: 65.625000] [G loss: 0.9252617359161377]
29751 [D loss: 0.6469165086746216, acc_real: 59.375000, acc_fake: 62.500000] [G loss: 0.9893790483474731]
29752 [D loss: 0.641311526298523, acc_real: 65.625000, acc_fake: 68.750000] [G loss: 0.963232696056366]
29753 [D loss: 0.5846246480941772, acc_real: 62.500000, acc_fake: 84.375000] [G loss: 1.0398893356323242]
29754 [D loss: 0.6500691175460815, acc_real: 68.750000, acc_fake: 50.000000] [G loss: 1.0065263509750366]
29755 [D loss: 0.6413507461547852, acc_real: 56.

29827 [D loss: 0.5922545790672302, acc_real: 62.500000, acc_fake: 87.500000] [G loss: 1.0530824661254883]
29828 [D loss: 0.632125198841095, acc_real: 71.875000, acc_fake: 53.125000] [G loss: 1.047954797744751]
29829 [D loss: 0.595468282699585, acc_real: 53.125000, acc_fake: 81.250000] [G loss: 1.0263925790786743]
29830 [D loss: 0.6326421499252319, acc_real: 56.250000, acc_fake: 65.625000] [G loss: 1.0915875434875488]
29831 [D loss: 0.6141570210456848, acc_real: 46.875000, acc_fake: 84.375000] [G loss: 0.9799408912658691]
29832 [D loss: 0.6714040040969849, acc_real: 56.250000, acc_fake: 68.750000] [G loss: 0.9157401323318481]
29833 [D loss: 0.5900790095329285, acc_real: 56.250000, acc_fake: 75.000000] [G loss: 0.9387043714523315]
29834 [D loss: 0.6327036619186401, acc_real: 56.250000, acc_fake: 75.000000] [G loss: 0.9861915111541748]
29835 [D loss: 0.7026607990264893, acc_real: 53.125000, acc_fake: 53.125000] [G loss: 0.9004946947097778]
29836 [D loss: 0.6196295022964478, acc_real: 65.6

29907 [D loss: 0.6989350318908691, acc_real: 59.375000, acc_fake: 50.000000] [G loss: 0.9247732758522034]
29908 [D loss: 0.5802396535873413, acc_real: 71.875000, acc_fake: 81.250000] [G loss: 0.9741071462631226]
29909 [D loss: 0.6671012043952942, acc_real: 56.250000, acc_fake: 71.875000] [G loss: 0.947388768196106]
29910 [D loss: 0.5382641553878784, acc_real: 78.125000, acc_fake: 81.250000] [G loss: 0.926711916923523]
29911 [D loss: 0.6472529172897339, acc_real: 53.125000, acc_fake: 56.250000] [G loss: 1.0059478282928467]
29912 [D loss: 0.6043932437896729, acc_real: 65.625000, acc_fake: 65.625000] [G loss: 0.9651564359664917]
29913 [D loss: 0.6535130739212036, acc_real: 62.500000, acc_fake: 56.250000] [G loss: 0.9380701780319214]
29914 [D loss: 0.6875505447387695, acc_real: 59.375000, acc_fake: 56.250000] [G loss: 0.9259678721427917]
29915 [D loss: 0.657489538192749, acc_real: 65.625000, acc_fake: 59.375000] [G loss: 0.9792118072509766]
29916 [D loss: 0.6456801295280457, acc_real: 53.1

29985 [D loss: 0.5404547452926636, acc_real: 84.375000, acc_fake: 68.750000] [G loss: 1.1096692085266113]
29986 [D loss: 0.6604528427124023, acc_real: 40.625000, acc_fake: 71.875000] [G loss: 1.003153681755066]
29987 [D loss: 0.664866030216217, acc_real: 56.250000, acc_fake: 53.125000] [G loss: 0.9965342283248901]
29988 [D loss: 0.5801978707313538, acc_real: 62.500000, acc_fake: 78.125000] [G loss: 1.0419068336486816]
29989 [D loss: 0.5942115187644958, acc_real: 65.625000, acc_fake: 59.375000] [G loss: 1.0635719299316406]
29990 [D loss: 0.6324573159217834, acc_real: 50.000000, acc_fake: 68.750000] [G loss: 1.0697226524353027]
29991 [D loss: 0.6466739177703857, acc_real: 53.125000, acc_fake: 65.625000] [G loss: 1.013338327407837]
29992 [D loss: 0.6020708084106445, acc_real: 68.750000, acc_fake: 65.625000] [G loss: 0.9853633642196655]
29993 [D loss: 0.634098470211029, acc_real: 59.375000, acc_fake: 78.125000] [G loss: 1.0140790939331055]
29994 [D loss: 0.5864827632904053, acc_real: 59.37